##Practical Example of BPNN

credit: https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/

In [1]:
from random import seed
from random import random

from math import exp
import numpy as np


In [2]:
# Input Data
w = np.array([0.5, 0.3])
bias = 0.1
alpha = 0.5
Epoch = 2
momentum = 0.7

In [3]:
 # Initialize a network, and randomly assign the weights
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

network = initialize_network(2, 1, 1)
for layer in network:
	print(layer)

[{'weights': [0.0258151356728199, 0.7577006112981947, 0.47513778239681204]}]
[{'weights': [0.020925474736906757, 0.22337725207287196]}]


In [4]:
#manual assignment of weights
network = [[{'weights': [0.5, 0.3, 0.1]}],
 [{'weights': [0.6, 0.8]}]]

network

[[{'weights': [0.5, 0.3, 0.1]}], [{'weights': [0.6, 0.8]}]]

In [5]:
#Forward propagation


# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation

# Transfer neuron activation
def transfer(activation):
	return np.round(1.0 / (1.0 + exp(-activation)),2)

# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

# test forward propagation
row = [1, 2]
output = forward_propagate(network, row)
for layer in network:
	print(layer)

[{'weights': [0.5, 0.3, 0.1], 'output': 0.77}]
[{'weights': [0.6, 0.8], 'output': 0.78}]


In [6]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

# test backpropagation of error

expected = [1]
backward_propagate_error(network, expected)
for layer in network:
	print(layer)

[{'weights': [0.5, 0.3, 0.1], 'output': 0.77, 'delta': 0.004011527519999998}]
[{'weights': [0.6, 0.8], 'output': 0.78, 'delta': 0.03775199999999999}]


In [7]:
# Update network weights with error
def update_weights(network, row, l_rate, momentum):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] = neuron['weights'][j] + (l_rate * neuron['delta'] * inputs[j]) + (momentum * l_rate * neuron['delta'] * inputs[j-1])
            neuron['weights'][-1] = neuron['weights'][-1] + (l_rate * neuron['delta']) + (momentum * l_rate * neuron['delta'])


lr = 0.5
update_weights(network, row, lr, momentum)

for layer in network:
	print(layer)

[{'weights': [0.503409798392, 0.3, 0.103409798392], 'output': 0.77, 'delta': 0.004011527519999998}]
[{'weights': [0.624708684, 0.8320892000000001], 'output': 0.78, 'delta': 0.03775199999999999}]


In [10]:
#Apply this function to learn from data - you must provide the train variable which consist of your dataset

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate, momentum)
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [15]:
data = [1,1,0], [1,0,1], [0,1,1], [0,0,0]

n_inputs = len(data[0])-1
n_outputs = len(set([row[-1] for row in data]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, data, 0.5, 2, n_outputs)

for layer in network:
  print(layer)
  print('\n') 

>epoch=0, lrate=0.500, error=2.523
>epoch=1, lrate=0.500, error=2.385
[{'weights': [0.18542160751216127, 0.8836798176573013, 0.3286631605770476], 'output': 0.58, 'delta': 0.0005439120273909325}, {'weights': [0.08526852796790975, 0.8635608822567803, 0.7325510128252436], 'output': 0.67, 'delta': 0.004413389275355613}]


[{'weights': [0.5661854649996799, 0.22251928111309216, -0.013783018315365395], 'output': 0.58, 'delta': 0.10231200000000001}, {'weights': [0.26120752748849446, -0.09877672996443077, 0.40514708166449764], 'output': 0.67, 'delta': -0.148137}]


